---
title:  "혼자하는 Text분석_02"
excerpt: "Sckit-lean library 과 Keras를 사용한 Text 분석 비교"

categories:
  - Machine-Learning
tags:
  - ML
  - text anlysis
  - 머신러닝
last_modified_at: 2020-03-17T16:13:00-05:00
---

여기서는 우선, TF-IDF 에 대한 내용을 한번 더 상세히 다루고, n-gram , KoNLPy 순서로 진행한다.

(https://www.kaggle.com/c/word2vec-nlp-tutorial)
[참고로 TF-IDF 정리는 wikidocs 의 "딥 러닝을 이용한 자연어 처리 입문" 을 참고했다. 텍스트 분석용으로 한국어 책으로 훌륭하다.](https://wikidocs.net/31698)

In [ ]:
## 라이브러리 로드

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline
# 시각화 결과가 선명하게 표시되도록
%config InlineBackend.figure_format = 'retina'

## 시각화를 위한 한글폰트 설정

In [ ]:
# Window 한글폰트 설정
# plt.rc("font", family="Malgun Gothic")
# Mac 한글폰트 설정
plt.rc("font", family="AppleGothic")
plt.rc('axes', unicode_minus=False)

## IMDB 데이터 불러와서 활용하기

이미, 어느정도 정제되어 있어서....완벽한 예제라고 하기 어렵다....완전히 다시 다운받아서 해본다

In [7]:
## 다시 인터넷에서 다운 받고나서
from sklearn.datasets import load_files
reviews_train = load_files("D:/★2020_ML_DL_Project/Alchemy/dataset/aclImdb/train/")
text_train,y_train = reviews_train.data,reviews_train.target
%time

Wall time: 0 ns


In [8]:
print(type(text_train[0]))
print(text_train[0])

<class 'bytes'>
b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."


In [9]:
reviews_test = load_files("D:/★2020_ML_DL_Project/Alchemy/dataset/aclImdb/test/")
text_test,y_test = reviews_test.data,reviews_test.target
%time

Wall time: 0 ns


In [10]:
## <br /> tag 제거한다. 데이터 정제과정
text_train = [doc.replace(b"<br />",b" ") for doc in text_train]
text_test = [doc.replace(b"<br />",b" ") for doc in text_test]

In [11]:
print("text_train.shape :",len(text_train))
print("text_test.shape :",len(text_test))

text_train.shape : 25000
text_test.shape : 25000


## TF-IDF 집중 연습하기  
> 상단의 wikidocs 책을 바탕으로 했음을 밝힌다.!!